# SNN Event-Driven MLP MNIST 主流程

本Notebook为阶段2事件驱动反向传播算法的实现文件，在MNIST数据集上实现了一个简单的两层MLP结构的SNN，采用了事件驱动和激活驱动混合的实现，即在时间轴上做反向传播，只有在脉冲事件发生的时刻才真正“激活”梯
度更新。

## 1. 模块文件

- `network.py`：包含EventDrivenNeuron、LinearLayer、Network等神经网络相关类和函数。
- `losses.py`：包含SpikeLoss、loss_count、loss_kernel等损失函数。
- `stats.py`：包含learningStat、learningStats、EarlyStopping等工具类。
- `train_test.py`：包含train、test等训练和测试函数。
- `data_utils.py`：包含get_mnist等数据加载函数。

## 2. 主文件

本Notebook作为主入口，负责参数配置、模块导入、训练与测试流程控制、结果可视化等。

## 3. 导入自定义模块

In [1]:
import sys
from network import Network
from losses import SpikeLoss
from stats import LearningStats, EarlyStopping
from train_test import train, test
from data_utils import get_mnist
import torch
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import os

## 4. 数据加载与预处理

加载MNIST数据集，设置batch_size等参数，获得train_loader和test_loader。

In [2]:
# 配置数据参数
network_config = {
    "epochs": 10,
    "batch_size": 50,
    "n_steps": 5,
    "dataset": "MNIST",
    "lr": 0.0005,
    "loss": "softmax",
    "n_class": 10,
    "desired_count": 4,
    "undesired_count": 1,
    "tau_m": 5,
    "tau_s": 3,
    "model": "LIF"
}
data_path = os.path.expanduser("./MNIST")
train_loader, test_loader = get_mnist(data_path, network_config)

loading MNIST


## 5. 模型定义与初始化

配置`network_config`和`layer_config`，初始化Network、SpikeLoss、优化器等对象。

In [3]:
# 配置网络结构
layer_config = {
    'FC_1': {
        'type': 'linear',
        'n_inputs': 784,
        'n_outputs': 300,
        'weight_scale': 1,
        'threshold': 1
    },
    'FC_2': {
        'type': 'linear',
        'n_inputs': 300,
        'n_outputs': 10,
        'weight_scale': 1,
        'threshold': 1
    }
}

# 初始化模型、损失函数、优化器等
net = Network(network_config, layer_config, list(train_loader.dataset[0][0].shape)).cuda()
error = SpikeLoss(network_config).cuda()
optimizer = optim.AdamW(net.get_parameters(), lr=network_config['lr'], betas=(0.9, 0.999))
l_states = LearningStats()
early_stopping = EarlyStopping()

网络结构:
Linear层: FC_1, 输入形状: [1, 28, 28], 输出形状: [300, 1, 1], 权重形状: [300, 784]
Linear层: FC_2, 输入形状: [300, 1, 1], 输出形状: [10, 1, 1], 权重形状: [10, 300]
-----------------------------------------


## 6. 训练与测试循环

使用for循环调用train和test函数，记录每个epoch的训练和测试结果.

In [4]:
import copy
import gc
# 训练与测试主循环
loss_types = ['count', 'kernel', 'softmax']
all_stats = {}
for lt in loss_types:
    print(f"Starting training with loss type: {lt}")
    torch.cuda.empty_cache()
    curr_network_config = copy.deepcopy(network_config)
    curr_network_config['loss'] = lt
    # 重置模型参数
    net = Network(curr_network_config, layer_config, list(train_loader.dataset[0][0].shape)).cuda()
    error = SpikeLoss(curr_network_config).cuda()
    optimizer = optim.AdamW(net.get_parameters(), lr=curr_network_config['lr'], betas=(0.9, 0.999))
    l_states = LearningStats()
    early_stopping = EarlyStopping()

    from data_utils import init
    syn_a = init(curr_network_config['n_steps'], curr_network_config['tau_s'])
    best_acc = 0

    for epoch in range(curr_network_config['epochs']):
        l_states.training.reset()
        train(net, train_loader, optimizer, epoch, l_states, curr_network_config, layer_config, error, syn_a)
        l_states.training.update()
        l_states.testing.reset()
        test(net, test_loader, epoch, l_states, curr_network_config, layer_config, early_stopping, syn_a)
        l_states.testing.update()
        # if early_stopping.early_stop:
        #     break
    all_stats[lt] = copy.deepcopy(l_states)
    del net, error, optimizer, l_states, early_stopping
    gc.collect()
    torch.cuda.empty_cache()

Starting training with loss type: count
网络结构:
Linear层: FC_1, 输入形状: [1, 28, 28], 输出形状: [300, 1, 1], 权重形状: [300, 784]
Linear层: FC_2, 输入形状: [300, 1, 1], 输出形状: [10, 1, 1], 权重形状: [10, 300]
-----------------------------------------
Epoch 0: Train Accuracy: 87.778, Loss: 0.229
Epoch 0: Test Accuracy: 92.890
Epoch 1: Train Accuracy: 93.358, Loss: 0.159
Epoch 1: Test Accuracy: 93.180
Epoch 2: Train Accuracy: 94.018, Loss: 0.148
Epoch 2: Test Accuracy: 93.750
Epoch 3: Train Accuracy: 94.420, Loss: 0.141
Epoch 3: Test Accuracy: 94.000
Epoch 4: Train Accuracy: 94.740, Loss: 0.135
Epoch 4: Test Accuracy: 93.910
Epoch 5: Train Accuracy: 94.913, Loss: 0.132
Epoch 5: Test Accuracy: 93.700
Epoch 6: Train Accuracy: 95.122, Loss: 0.128
Epoch 6: Test Accuracy: 94.240
Epoch 7: Train Accuracy: 95.368, Loss: 0.126
Epoch 7: Test Accuracy: 94.000
Epoch 8: Train Accuracy: 95.467, Loss: 0.124
Epoch 8: Test Accuracy: 93.740
Epoch 9: Train Accuracy: 95.590, Loss: 0.122
Epoch 9: Test Accuracy: 94.120
Starting train

## 7. 可视化训练结果

绘制loss和accuracy曲线，并展示混淆矩阵图片。

In [5]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

def save_for_word_report(results, loss_types):
    # 设置全局绘图风格，使图表更专业
    plt.rcParams.update({'font.size': 12, 'font.family': 'sans-serif'})
    colors = {'count': '#D62728', 'kernel': '#1F77B4', 'softmax': '#2CA02C'}
    
    # 1. 训练损失对比图
    plt.figure(figsize=(8, 5))
    for lt in loss_types:
        plt.plot(results[lt].training.loss_log, label=f'Loss: {lt}', color=colors[lt], linewidth=2)
    plt.title('Training Loss Comparison', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Loss Value')
    plt.legend(frameon=True)
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.savefig('report_loss_comparison.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 2. 训练准确率对比图
    plt.figure(figsize=(8, 5))
    for lt in loss_types:
        plt.plot(results[lt].training.accuracy_log, label=f'Train Acc: {lt}', color=colors[lt], linewidth=2)
    plt.title('Training Accuracy Comparison', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend(frameon=True)
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.savefig('report_train_acc_comparison.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 3. 测试准确率对比图
    plt.figure(figsize=(8, 5))
    for lt in loss_types:
        plt.plot(results[lt].testing.accuracy_log, label=f'Test Acc: {lt}', color=colors[lt], linewidth=2.5)
    plt.title('Test Accuracy Comparison', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend(frameon=True)
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.savefig('report_test_acc_comparison.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 4. 混淆矩阵横向对比图 (合并成一张长图，方便 Word 占据一整行)
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    for i, lt in enumerate(loss_types):
        path = f'confusion_matrix_{lt}.png'
        if os.path.exists(path):
            img = mpimg.imread(path)
            axes[i].imshow(img)
            axes[i].set_title(f'Confusion matrix: {lt}', fontsize=12, fontweight='bold')
            axes[i].axis('off')
        else:
            axes[i].text(0.5, 0.5, 'Image Missing', ha='center')
            axes[i].axis('off')
    plt.tight_layout()
    plt.savefig('report_confusion_matrix_grid.png', dpi=300, bbox_inches='tight')
    plt.close()

    print("已生成四张报告专用图：")
    print("- report_loss_comparison.png (Loss对比)")
    print("- report_train_acc_comparison.png (训练精度对比)")
    print("- report_test_acc_comparison.png (测试精度对比)")
    print("- report_confusion_matrix_grid.png (混淆矩阵横向对比)")

# 调用
save_for_word_report(all_stats, loss_types)


已生成四张报告专用图：
- report_loss_comparison.png (Loss对比)
- report_train_acc_comparison.png (训练精度对比)
- report_test_acc_comparison.png (测试精度对比)
- report_confusion_matrix_grid.png (混淆矩阵横向对比)
